## DFT Course - 2022-23
## Beatriz Helena Cogollo-Olivo

## Tutorial 02 - The (very) basics of pw.x 

These examples cover some of the basic features of the PW package: scf, relax and vc-relax. In the first example we will search the lattice parameter of Si by two different approaches; and in the second one, we will compute the equilibrium geometry of a CO molecule.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Lattice parameter for silicon bulk with scf

$celldm(1)$ is the keyword that represents the lattice parameter, alat, (in a.u.) of a crystal structure.

In [ ]:
#Let's download the pseudopotential
!wget http://pseudopotentials.quantum-espresso.org/upf_files/Si.pz-vbc.UPF

In [ ]:
%%writefile Si-alat.sh

#!/bin/sh
NAME="alat"
for ALAT in AA BB CC DD #The ecut values are defined in this list
do
cat > ${NAME}_${ALAT}.in << EOF
 &control
    calculation = 'scf',
    prefix = 'silicon'
    outdir = './tmp/'
    pseudo_dir = './'
 /
 &system
    ibrav =  2,
    celldm(1) = $ALAT,
    nat =  2,
    ntyp = 1,
    ecutwfc = 40
 /
 &electrons
    mixing_beta = 0.6
 /
ATOMIC_SPECIES
 Si 28.086  Si.pz-vbc.UPF
ATOMIC_POSITIONS (alat)
 Si 0.0 0.0 0.0
 Si 0.25 0.25 0.25
K_POINTS (automatic)
  6 6 6 1 1 1
EOF

pw.x < ${NAME}_${ALAT}.in > ${NAME}_${ALAT}.out
echo ${NAME}_${ALAT}
grep ! ${NAME}_${ALAT}.out

done

In [ ]:
!chmod +x Si-alat.sh
!sh Si-alat.sh

In [ ]:
x1 = [] #Include the alat values separated by commas
y1 = [] #Include the total energy values separated by commas
plt.xlabel('Alat (a.u.)')
plt.ylabel('Total energy (Ry)')
plt.plot(x1, y1, '-o')
plt.show()

#### Question 1
What is the value of alat that minimizes the energy?

### Lattice parameter for silicon bulk with vc-relax

We can run a more reliable test, by changing the type of calculation to a variable-cell relaxation that changes the lattice parameter until the lowest energy is found.

In [ ]:
%%writefile Si-vc-relax.in

 &control
    calculation = 'vc-relax',
    prefix = 'silicon'
    outdir = './tmp/'
    pseudo_dir = './'
 /
 &system
    ibrav =  2,
    celldm(1) = AA, #Set an initial alat value (not too) far from equilibrium
    nat =  2,
    ntyp = 1,
    ecutwfc = 40
 /
 &electrons
    mixing_beta = 0.6
 /
 &ions
    ion_dynamics = 'bfgs'
 /
 &cell
    cell_dynamics = 'bfgs'
 /
ATOMIC_SPECIES
 Si 28.086  Si.pz-vbc.UPF
ATOMIC_POSITIONS (alat)
 Si 0.0 0.0 0.0
 Si 0.25 0.25 0.25
K_POINTS (automatic)
  6 6 6 1 1 1

In [ ]:
!pw.x < Si-vc-relax.in > Si-vc-relax.out

In [ ]:
#Let's see how the structure changes
!xcrysden --pwo Si-vc-relax.out

In [ ]:
!grep ! Si-vc-relax.out

#### Question 2
How many scf steps were required for optimizing the structure? (The last energy value correspond to the final scf calculation at the relaxed structure, so it should not be taken into account).

In [ ]:
x2 = [] #Include the scf steps separated by commas
y2 = [] #Include the total energy values separated by commas
plt.xlabel('SCF steps')
plt.ylabel('Total energy (Ry)')
plt.plot(x2, y2)
plt.show()

In [ ]:
new_y2 = [(i+ENERGY) for i in y2] #Sum the lowest energy value
plt.xlabel('SCF steps')
plt.ylabel('Total energy difference (meV/atom)')
plt.ylim(0, LIM) #The y-axis limits can be changed here
plt.plot(x2, new_y2)
plt.grid()
plt.show()

#### Question 3
What is the energy difference between the last two scf steps? Is it in agreement with the value reported in the output file?

In [ ]:
!grep "Begin" -A 15 Si-vc-relax.out

#### Question 4
For ibrav=2, the cell is defined as follows:

v1 = (a/2)(-1,0,1),  v2 = (a/2)(0,1,1), v3 = (a/2)(-1,1,0)

What is the optimized alat value for Si? Is it consistent with the first part of the exercise?

### Geometry relaxation for CO molecule

In [ ]:
#Let's download the pseudopotentials
!wget https://pseudopotentials.quantum-espresso.org/upf_files/C.pz-rrkjus.UPF
!wget https://pseudopotentials.quantum-espresso.org/upf_files/O.pz-rrkjus.UPF

In [ ]:
%%writefile CO-relax.in

 &control
    calculation = 'relax',
    prefix = 'CO'
    outdir = './tmp/'
    pseudo_dir = './'
 /
 &system
    ibrav =  0,
    nat =  2,
    ntyp = 2,
    ecutwfc = 40,
    ecutrho = 400
 /
 &electrons
    conv_thr    = 1.D-7,
    mixing_beta = 0.7
 /
 &ions
    ion_dynamics = 'bfgs'
 /
CELL_PARAMETERS bohr
12.0  0.0  0.0
 0.0 12.0  0.0
 0.0  0.0 12.0
ATOMIC_SPECIES
O  1.00  O.pz-rrkjus.UPF
C  1.00  C.pz-rrkjus.UPF
ATOMIC_POSITIONS {bohr}
C  2.256  0.0  0.0
O  0.000  0.0  0.0  0 0 0
K_POINTS {Gamma}

#### Question 5
What is the CO bond length before the relaxation?

In [ ]:
#Let's see how the molecule looks before the relaxation
!xcrysden --pwi CO-relax.in

In [ ]:
!pw.x < CO-relax.in > CO-relax.out

In [ ]:
#Let's see how the molecule changes after the relaxation
!xcrysden --pwo CO-relax.out

#### Question 6
The experimental value of the CO bond length is 0.112830 nm. Is this value consistent with the bond length obtained after the relaxation?